# Coastal Flood Protection

The Mayors Office of Climate Resiliency published a document in December 2021.  I will use this as the basis to integrate NYC geodata and STEW-MAP.

   - Existing Conditions
   - Neighborhood Risk
   - Stewardship for Coastal Protection

In [ ]:
IFrame('../references/Coastal-Protection-Guidance.pdf', width=1200, height=1000)

# Existing Conditions

In [ ]:
brooklyn_gdf = gpd.read_parquet('../data/processed/admin-boundaries/brooklyn.parq')

In [ ]:
brooklyn_turfs_gdf = gpd.read_parquet('../data/processed/brooklyn/brooklyn-turfs.parq')

In [ ]:
brooklyn_turf_subset_gdf = brooklyn_turfs_gdf.sort_values('Shape_Area')[['OrgName', 'OrgWebSite', 'PrimST', 'PopID', 'geometry']][0:225]#.explore()

In [ ]:
brooklyn_extreme_flooding_gdf = gpd.read_parquet('../data/processed/DEP/brooklyn-extreme.parq')

In [ ]:
center = brooklyn_gdf.iloc[0].geometry.centroid.y, brooklyn_gdf.iloc[0].geometry.centroid.x

In [ ]:
center

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True

google_map = TileLayer(
    url="https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Maps",
)
google_map.base = True

google_satellite = TileLayer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    attribution="Google",
    name="Google Satellite"
)
google_satellite.base = True

map_display = Map(center=center, zoom=12,
                  layers=[google_satellite, google_map, imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

map_display.add_control(LayersControl())

map_display.add_control(FullScreenControl())

map_display

In [ ]:
brooklyn_boundary = GeoData(geo_dataframe = brooklyn_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Brooklyn')

map_display += brooklyn_boundary

In [ ]:
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange', 'purple', 'blue']),
    }

import json
import random
turfs_geojson = brooklyn_turf_subset_gdf.to_json()

geo_json = GeoJSON(
    data=json.loads(turfs_geojson),
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.5, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.8
    },
    style_callback=random_color,
    name='turfs geojson'
)

map_display.add_layer(geo_json)

turf_html = HTML('''Hover over a turf''')
turf_html.layout.margin = '0px 20px 20px 20 px'
turf_control = WidgetControl(widget=turf_html, position='bottomright')

def update_turf_html(feature, **kwargs):
    turf_html.value = f"<b>Name: {feature['properties']['OrgName']}<br><b>Primary: {feature['properties']['PrimST']}"
    
map_display.add_control(turf_control)  # does += work for this?

geo_json.on_hover(update_turf_html)

In [ ]:
#turfs = GeoData(geo_dataframe = brooklyn_turf_subset_gdf,
#                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
#                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
#                   name = 'Turfs')
#
#map_display += turfs

In [ ]:
flood = GeoData(geo_dataframe = brooklyn_extreme_flooding_gdf,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Extreme Flood')

map_display += flood

In [ ]:
dep_311_2021_gdf = gpd.read_parquet('../data/processed/DEP/2021-311.parq')

In [ ]:
len(dep_311_2021_gdf)

In [ ]:
dep_311_2021_gdf.iloc[27]['Created Date'].day

In [ ]:
dep_311_2021_gdf['day'] = dep_311_2021_gdf['Created Date'].apply(lambda dt: dt.date().strftime("%m-%d (%A)"))

In [ ]:
foo = dep_311_2021_gdf[dep_311_2021_gdf.Location.notnull()]

In [ ]:
len(foo)

In [ ]:
len(dep_311_2021_gdf)

# 311

In [ ]:
import folium
from folium import plugins

In [ ]:
hmap = folium.Map(location = center, tiles='Stamen Toner', zoom_start = 10)
plugins.Fullscreen().add_to(hmap);

In [ ]:
from collections import defaultdict, OrderedDict

data = defaultdict(list)
for row in foo.itertuples():
    data[row.day].append([row.geometry.y, row.geometry.x])
    #try:
        #data[row.day].append([row.geometry.y, row.geometry.x])
    #except:
        #pass
    #if row.Location is not None:
        #data[row.day].append([row.geometry.y, row.geometry.x])
    
data = OrderedDict(sorted(data.items(), key=lambda t: t[0]))

In [ ]:
hm = plugins.HeatMapWithTime(data=list(data.values()),
                     index=list(data.keys()), 
                     
                     radius=10,
                     auto_play=True,
                     max_opacity=0.8)

In [ ]:
hm.add_to(hmap)
hmap